In [11]:
import numpy as np
import pandas as pd
import pickle
import os
from lib.read_data import dataset,Datasets,readDataWithRawScore
from math import copysign
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
# plot
import matplotlib.pyplot as plt
% matplotlib inline

PROJECT_DIR = '/home/shuang/projects/eqtm/projector_withNonLinear'

In [2]:
def generate_metadata(labels,save_path):
    with open(save_path,'w') as f:
        for i in range(labels.shape[0]):
            f.write('{}\n'.format(int(labels.loc[i,])))

In [5]:
def generate_embedding(datapoints):
    sess = tf.InteractiveSession()
    with tf.device("/cpu:0"):
        embedding = tf.Variable(tf.stack(datapoints,axis=0),
                                trainable=False,
                                name='embedding')
    tf.global_variables_initializer().run()
    saver = tf.train.Saver()
    writer = tf.summary.FileWriter(PROJECT_DIR, sess.graph)
    
    config = projector.ProjectorConfig()
    embed= config.embeddings.add()
    embed.tensor_name = 'embedding:0'
    embed.metadata_path = os.path.join(PROJECT_DIR,'metadata.tsv')
    
    projector.visualize_embeddings(writer, config)
    saver.save(sess,
               os.path.join(PROJECT_DIR,'a_model.ckpt'),
               global_step=datapoints.shape[0])
    

In [9]:
if __name__=='__main__':
    
#     def load_bonderWestraData(path,test_size=0.25):
#         data = pd.read_csv(path,sep=',',index_col=0)
#         def binarize(row):
#             if row > 0:
#                 return 1
#             else:
#                 return 0
#         data['direction'] = data['zscore'].apply(binarize)
#         print('Raw data loaded.')
#         dataset = readDataWithRawScore(data,'direction',test_size=test_size)
#         return dataset
    
    bonder_path = 'data/bonder_withZscoreAndNonLinear.csv'
    data = pd.read_csv(bonder_path)
    bonder = readDataWithRawScore(data,'direction',test_size=0)

    print('Bonder dataset loaded.',bonder.train.values.shape)

Data Normalized.
Bonder dataset loaded. (2842, 58)


In [12]:
    labels = bonder.train.labels
    save_path = os.path.join(PROJECT_DIR,'metadata.tsv')
    generate_metadata(labels,save_path)
    datapoints = np.array(bonder.train.values[[col for col in bonder.train.values.columns if col not in ['cpgName']]])
    generate_embedding(datapoints)